### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [15]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.G_spec_fun = self.__gini_spec
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.G_spec_fun = self.__entropy_spec
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.G_spec_fun = self.__misclass_spec
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - (((l_c + r_c) / (l_s + r_s)) ** 2 ).sum(axis = 1) -\
        ((1 - l_c ** 2 / l_s + 1 - r_c ** 2 / r_s) /  (l_s + r_s)).sum(axis=1) 
    
    def __gini_spec(self, y_c, y_s):
        return 1 - ((np.bincount(y_c).astype(np.float) / y_s) ** 2).sum()
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return -((l_c + r_c) / (l_s + r_s) * np.log((l_c + r_c) / (l_s + r_s))).sum(axis = 1) +\
        ((l_c * np.log(l_c / l_s) + r_c * np.log(r_c / r_c)) / (l_s + r_s)).sum(axis=1)
    
    def __entropy_spec(self, l_c, l_s, r_c, r_s):
        return -np.sum(np.bincount(y).astype(np.float) / y.size *\
        np.log(np.bincount(y).astype(np.float) / y.size))
    
    def __misclass(self, l_c, l_s, r_c, r_s):
        return -np.max((l_c + r_c) / (l_s + r_s), axis = 1) +\
        (np.max(l_c / (r_s + l_s), axis = 1) + np.max(r_c / (r_s + l_s), axis = 1)) - 1
                 
    def __misclass_spec(self, l_c, l_s, r_c, r_s):
        return 1 - np.max(np.bincount(y).astype(np.float) / y.size)
    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:round(np.sqrt(n_feature))])
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:round(np.log2(n_feature))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        borders = np.where(sorted_y[:-1] != sorted_y[1:])[0]
        if len(borders) == 0:
            return np.inf, None
        #сколько одинаковых подряд идет
        equals = borders - np.append(np.array([-1]), borders[:-1])
        #какому классу принадлежит отрезок
        what_class_rules_the_range = np.zeros((equals.shape[0],class_number))
        what_class_rules_the_range[np.arange(equals.shape[0]), sorted_y[borders]] = 1
        increments = what_class_rules_the_range * equals.reshape(-1, 1)
        
        l_class_count = np.cumsum(increments, axis = 0)
        r_class_count = np.bincount(sorted_y, minlength = class_number) - l_class_count
        
        l_sizes = borders.reshape(l_class_count.shape[0], 1) + 1
        r_sizes = sorted_y.shape[0] - l_sizes 
        weights = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmax(weights)
        element_id = l_sizes[idx][0]
        return weights[idx], (sorted_x[element_id-1] + sorted_x[element_id]) / 2.0
    
    def __fit_node(self, x, y, node_id, depth):
        #старые значения классов
        f = np.array(y)
        if self.max_depth != None and depth >= self.max_depth or\
                    y.shape[0] < self.min_samples_split or\
                    self.sufficient_share <= np.bincount(y).max() / y.size:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(),
                                  np.bincount(y).astype(np.float) / y.size)
        #отображает множество значений классов в множество вида [1..N]
        t = dict()
        i = 0
        for j in range(y.shape[0]):
            if y[j] not in t.keys():
                t.update({y[j]:i})
                y[j] = i
                i += 1
            else:
                y[j] = t[y[j]]
                
        ids = np.arange(x.shape[1])
        thresholds = np.array([self.__find_threshold(x[:, idx], y) for idx in range(x.shape[1])])
        best = thresholds[:, 0].argmax()
        best_thresh = thresholds[best, 1]
        
        if best_thresh is None:
            tmp = np.bincount(f)
            self.tree[node_id] = (self.LEAF_TYPE, tmp.argmax(), tmp.astype(np.float) / y.size)
            return
        
        x_l, x_r, y_l, y_r = self.__div_samples(x, f, ids[best], best_thresh)
        if y_l.shape[0] == 0 or y_r.shape[0] == 0:
            tmp = np.bincount(y)
            self.tree[node_id] = (self.LEAF_TYPE, tmp.argmax(), tmp.astype(np.float) / y.size)
            return
        
        self.tree[node_id] = (self.NON_LEAF_TYPE, ids[best], best_thresh)
        self.feature_importances_[best] += self.G_spec_fun(y, y.shape[0]) - y_l.shape[0] / y.shape[0] * \
        self.G_spec_fun(y_l, y_l.shape[0]) - y_r.shape[0] / y.shape[0] * self.G_spec_fun(y_r, y_r.shape[0])
        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)
        return 
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1], dtype=np.float)
        self.__fit_node(x, y, 0, 0) 
        self.feature_importances_ /= self.feature_importances_.sum()
        
    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [16]:
my_clf = MyDecisionTreeClassifier(min_samples_split = 2)
clf = DecisionTreeClassifier(min_samples_split = 2)

In [17]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [18]:
%time clf.fit(X_train, y_train)

CPU times: user 6.61 ms, sys: 806 µs, total: 7.42 ms
Wall time: 4.12 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [19]:
%time my_clf.fit(X_train, y_train)

CPU times: user 30.6 ms, sys: 642 µs, total: 31.2 ms
Wall time: 27.5 ms


## Проверка качества работы на wine

In [20]:
f1_score(y_pred = clf.predict(X_test), y_true = y_test, average='macro')

0.8311688311688311

In [21]:
f1_score(y_pred = my_clf.predict(X_test), y_true = y_test, average='macro')

0.9474747474747475

## Подготовка данных Speed Dating Data 

In [22]:
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')

In [23]:
to_drop = ['age_o', 'race_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'dec_o', 'attr_o',
              'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o',
             'prob_o', 'met_o',
           'partner','order','position', 'positin1',
               'round', 'condtn', 'idg', 'id',  
           'sports', 'tvsports', 
           'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming',
           'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts',
           'music', 'shopping', 'yoga', 'expnum', 'wave']
           

In [24]:
df = df.iloc[:, :95]

df = df.drop(to_drop, axis = 1)

In [25]:
info = ['field_cd', 'mn_sat', 'tuition', 'income', 'career_c' ]

In [26]:
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)

df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)

df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)

df.loc[:,info] = df.loc[:,info].fillna(0)

df_he = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
            .drop(['gender'], axis = 1).dropna()

df_she = df.query('gender == 0').drop_duplicates(subset=['iid']) \
            .drop(['gender', 'match', 'int_corr', 'samerace'], axis = 1).dropna()

df_she.columns = df_she.columns + '_s'
df_she = df_she.drop(['pid_s'], axis = 1)
df_pair = df_he.join(df_she.set_index('iid_s'), on = 'pid', how = 'inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)

features_names = df_pair.columns[1:]


In [27]:
for j in df_pair.columns:
    k = df_pair[j].isnull().sum()
    if  type(df_pair[j][df_pair[j].first_valid_index()]) is str:
        df_pair = df_pair.drop([j], axis = 1)

In [28]:
df_pair

,match,int_corr,samerace,age,field_cd,mn_sat,tuition,race,imprace,imprelig,...,amb2_1_s,shar2_1_s,attr3_1_s,sinc3_1_s,fun3_1_s,intel3_1_s,amb3_1_s,attr5_1_s,sinc5_1_s,intel5_1_s
3507,0,0.05,0,32.0,7.0,1430.0,26908.0,2.0,1.0,6.0,...,10.0,10.0,7.0,7.0,8.0,8.0,10.0,9.0,6.0,9.0
3516,0,0.44,0,25.0,10.0,1140.0,9168.0,2.0,7.0,1.0,...,10.0,10.0,7.0,7.0,8.0,8.0,10.0,9.0,6.0,9.0
3525,0,-0.20,0,25.0,10.0,0.0,0.0,4.0,1.0,1.0,...,10.0,10.0,7.0,7.0,8.0,8.0,10.0,9.0,6.0,9.0
3534,0,0.23,1,27.0,8.0,1360.0,26062.0,6.0,1.0,1.0,...,10.0,10.0,7.0,7.0,8.0,8.0,10.0,9.0,6.0,9.0
3508,0,0.43,1,32.0,7.0,1430.0,26908.0,2.0,1.0,6.0,...,10.0,20.0,6.0,9.0,9.0,8.0,9.0,6.0,9.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8245,0,0.13,0,30.0,8.0,0.0,0.0,2.0,3.0,4.0,...,10.0,20.0,3.0,9.0,9.0,9.0,7.0,3.0,9.0,9.0
8289,0,0.23,0,30.0,8.0,0.0,0.0,2.0,8.0,8.0,...,10.0,20.0,3.0,9.0,9.0,9.0,7.0,3.0,9.0,9.0
8311,0,0.29,0,28.0,8.0,0.0,0.0,2.0,2.0,3.0,...,10.0,20.0,3.0,9.0,9.0,9.0,7.0,3.0,9.0,9.0
8355,0,-0.32,0,27.0,8.0,1400.0,26019.0,1.0,2.0,1.0,...,10.0,20.0,3.0,9.0,9.0,9.0,7.0,3.0,9.0,9.0


In [29]:
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values
features_names = df_pair.columns[1:]

In [30]:
clf = DecisionTreeClassifier()
my_clf = MyDecisionTreeClassifier()

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.4, random_state=42)

In [32]:
np.unique(y_test, return_counts = True)

(array([0, 1]), array([597,  99]))

In [33]:
np.unique(y_train, return_counts = True)

(array([0, 1]), array([873, 171]))

## Проверка скорости работы на Speed Dating Data 

In [48]:
%time clf.fit(X_train, y_train)


CPU times: user 29.4 ms, sys: 17 µs, total: 29.4 ms
Wall time: 27.7 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [35]:
%time my_clf.fit(X_train, y_train)

CPU times: user 261 ms, sys: 470 µs, total: 262 ms
Wall time: 259 ms


## Проверка качества работы на Speed Dating Data

In [49]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5213728646671772

In [37]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5369031540478271

## Задание 3

## Задание 4

In [38]:
imp = np.argpartition(clf.feature_importances_, -10)[-10:]

In [39]:
clf.feature_importances_[imp]

array([0.02283022, 0.02358793, 0.02439176, 0.02779535, 0.02455049,
       0.03298889, 0.03781866, 0.03545336, 0.03495655, 0.09502328])

In [40]:
my_imp = np.argpartition(my_clf.feature_importances_, -10)[-10:]

In [41]:
my_clf.feature_importances_[my_imp]

array([0.02224843, 0.11146861, 0.06571816, 0.06690328, 0.0818953 ,
       0.06499627, 0.0451363 , 0.13270072, 0.09952554, 0.265254  ])

In [42]:
for i in imp:
    print(clf.feature_importances_[i],' ',features_names[i])

0.022830224844819554   shar1_1
0.023587934440758306   amb1_1_s
0.02439175602496279   shar1_1_s
0.02779534660525798   amb3_1_s
0.024550494219512126   amb4_1_s
0.03298889268064489   age_s
0.03781866236079214   sinc1_1_s
0.0354533576226882   income_s
0.03495654683934662   exphappy_s
0.09502328452126206   int_corr


In [43]:
for i in my_imp:
    print(my_clf.feature_importances_[i],' ',features_names[i])

0.022248434428592555   shar1_1
0.11146860591347865   fun4_1_s
0.0657181631373654   intel1_1
0.06690328033497481   imprelig_s
0.08189530230378021   tuition_s
0.06499627166966686   field_cd_s
0.04513629977060201   fun2_1
0.13270072132556976   samerace
0.09952554099417736   age
0.2652539974052223   int_corr


## Задание 5

In [46]:
from sklearn.ensemble import RandomForestClassifier

In [47]:
params = {
    'n_estimators':range(1, 1000,100),
    'max_depth':range(1,20),
    'min_samples_leaf': range(1,20, 1),
}

rf_clf = RandomForestClassifier()
find_best = RandomizedSearchCV(rf_clf, param_distributions = params, cv = 5)
find_best.fit(X_train, y_train)
print(find_best.best_params_)

{'n_estimators': 901, 'min_samples_leaf': 12, 'max_depth': 5}
